## Take the basic toml config file, and make iterations

In [21]:
import toml
import os
# from transformers import AutoModel, BertModel, AutoModelForMaskedLM



In [22]:
source = "/home/egil/gits_wsl/wl-coref-ncc/config.toml"
toml_folder = "/home/egil/gits_wsl/wl-coref-ncc/tomls"
train_path = "/home/egil/gits_wsl/ncc/wl-ncc_heads/wl-ncc_train_head.jsonl"
runpath = "/home/egil/gits_wsl/wl-coref-ncc/run.py"

with open(source) as rf:
    base_toml ={'DEFAULT': toml.loads(rf.read())['DEFAULT']}

defaults = base_toml['DEFAULT']
for key, value in defaults.items():
    print(key,":\t", value)




data_dir :	 data
train_data :	 data/english_train_head.jsonlines
dev_data :	 data/english_development_head.jsonlines
test_data :	 data/english_test_head.jsonlines
device :	 cuda:0
bert_model :	 bert-large-cased
bert_window_size :	 512
embedding_size :	 20
sp_embedding_size :	 64
a_scoring_batch_size :	 512
hidden_size :	 1024
n_hidden_layers :	 1
max_span_len :	 64
rough_k :	 50
bert_finetune :	 True
dropout_rate :	 0.3
bert_learning_rate :	 1e-05
learning_rate :	 0.0003
train_epochs :	 20
bce_loss_weight :	 0.5
conll_log_dir :	 data/conll_logs
tokenizer_kwargs :	 {'roberta-large': {'add_prefix_space': True}, 'spanbert-large-cased': {'do_lower_case': False}, 'bert-large-cased': {'do_lower_case': False}}


## Changes from the default that is shared by all experiments go here

In [23]:
run_id = "models01"

defaults["device"] = "cpu"
defaults["bert_finetune"] = False
defaults["train_epochs"] = 2
defaults["train_data"] = train_path
defaults["dev_data"] = train_path.replace("train", "development")
defaults["test_data"] = train_path.replace("train", "test")
out_folder = os.path.join(toml_folder, run_id)
defaults["conll_log_dir"] = os.path.join(out_folder, "conll_logs")
defaults["data_dir"] = out_folder
if not os.path.exists(out_folder):
    os.mkdir(out_folder)
    if not os.path.exists(defaults["conll_log_dir"]):
        os.mkdir(defaults["conll_log_dir"])


## Create lists of what should be iterated over, and write a toml file with each of these experiments in the one file
Start without any grid, list only

In [24]:
alternatives = {"bert_models": ["xlm-roberta-base", "bert-base-multilingual-cased","/home/egil/datasets/norbert2", "/home/egil/datasets/nb-bert-base",]}
exp_ids = []
out_toml = {'DEFAULT': defaults}
for key, alts in alternatives.items():
    param_name = key[:-1] # Always add s also when s from before
    for idx, alt in enumerate(alts):
        experiment_id = run_id+"_"+str(idx).zfill(3)
        out_toml[experiment_id] = {param_name: alt}
        exp_ids.append(experiment_id)
out_toml_path = os.path.join(toml_folder, run_id+".toml")
with open(out_toml_path, "w") as wf:
    toml.dump(out_toml, wf)


## Norwegian models
Got them to run with cloned and local path, and torch=1.6 in stead of 1.4

In [25]:
# Check we can find the models:

# am_lms = ["ltgoslo/norbert2", "NbAiLab/nb-bert-base", "flax-community/roberta-base-scandinavian" ]
for model in []: # alternatives["bert_models"]:
    try:
        m = AutoModel.from_pretrained(model).to("cpu")
        print("success", model)
    except OSError as e:
        print("failure", model)
        # print(e, "\n")

# model = "/home/egil/datasets/norbert2"
# model = "/home/egil/datasets/nb-bert-base"
# m = AutoModel.from_pretrained(model, local_files_only=True).to("cpu")

## Create script


In [26]:

script_path = os.path.join(toml_folder, run_id+".sh")
scriptlines  = ["#!/bin/sh"]
for exp in exp_ids:
    scriptlines.append(" ".join(["python", runpath, "train", exp, "--config-file", out_toml_path]))

with open (script_path, "w") as wf:
    wf.write("\n".join(scriptlines))
print(script_path)
print(runpath)
print("\n".join(scriptlines))


/home/egil/gits_wsl/wl-coref-ncc/tomls/models01.sh
/home/egil/gits_wsl/wl-coref-ncc/run.py
#!/bin/sh
python /home/egil/gits_wsl/wl-coref-ncc/run.py train models01_000 --config-file /home/egil/gits_wsl/wl-coref-ncc/tomls/models01.toml
python /home/egil/gits_wsl/wl-coref-ncc/run.py train models01_001 --config-file /home/egil/gits_wsl/wl-coref-ncc/tomls/models01.toml
python /home/egil/gits_wsl/wl-coref-ncc/run.py train models01_002 --config-file /home/egil/gits_wsl/wl-coref-ncc/tomls/models01.toml
python /home/egil/gits_wsl/wl-coref-ncc/run.py train models01_003 --config-file /home/egil/gits_wsl/wl-coref-ncc/tomls/models01.toml


In [27]:
# sudo apt-get install git-lfs
# git lfs install
# git clone https://huggingface.co/ltgoslo/norbert2
